In [32]:
!pip install numpy
!pip install scikit-image
!pip install imutils
!pip install scikit-learn
!pip install opencv-contrib-python
!pip install Flask
!sudo apt update
!sudo apt upgrade
!pip install opencv-python
!pip install opencv-python-headless

/usr/bin/sh: sudo: command not found
/usr/bin/sh: sudo: command not found


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import cv2
import os
import pickle
from io import BytesIO
import zipfile
import matplotlib.pyplot as plt

In [4]:
def quantify_image(image):
	# compute the histogram of oriented gradients feature vector for
	# the input image
	features = feature.hog(image, orientations=9,
		pixels_per_cell=(10, 10), cells_per_block=(2, 2),
		transform_sqrt=True, block_norm="L1")
	# return the feature vector
	return features

In [5]:
def load_split(path):
	# grab the list of images in the input directory, then initialize
	# the list of data (i.e., images) and class labels
	imagePaths = list(paths.list_images(path))
	data = []
	labels = []
	# loop over the image paths
	for imagePath in imagePaths:
		# extract the class label from the filename
		label = imagePath.split(os.path.sep)[-2]
		# load the input image, convert it to grayscale, and resize
		# it to 200x200 pixels, ignoring aspect ratio
		image = cv2.imread(imagePath)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		image = cv2.resize(image, (200, 200))
		# threshold the image such that the drawing appears as white
		# on a black background
		image = cv2.threshold(image, 0, 255,
			cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
		# quantify the image
		features = quantify_image(image)
		# update the data and labels lists, respectively
		data.append(features)
		labels.append(label)
	# return the data and labels
	return (np.array(data), np.array(labels))

In [11]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='sGtuz_dFiWDs1nMorkYkb95xDRTv_JZ7FJbNB-3BPlfa',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'diseasedetection-donotdelete-pr-tdnyhkkq8hbqlb'
object_key = 'spiral.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [12]:

unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [15]:
!pwd
!ls

/home/wsuser/work
spiral


In [16]:
trainingPath = r"/home/wsuser/work/spiral/training"
testingPath = r"/home/wsuser/work/spiral/testing"

loading the training and testing data

In [17]:
print("[INFO] loading data...")
(X_train, y_train) = load_split(trainingPath)
(X_test, y_test) = load_split(testingPath)

[INFO] loading data...


In [18]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
print(X_train.shape,y_train.shape)
print("[INFO] training model")
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
pickle.dump(model,open('parkinson.pkl','wb'))
predictions = model.predict(X_test)

(72, 12996) (72,)
[INFO] training model


In [19]:
cm = confusion_matrix(y_test, predictions).flatten()
print(cm)
(tn, fp, fn, tp) = cm
accuracy = (tp + tn) / float(cm.sum())
print(accuracy)

[14  1  3 12]
0.8666666666666667


randomly select a few images and then initialize the output images for the montage

In [20]:
testingPaths = list(paths.list_images(testingPath))
idxs = np.arange(0, len(testingPaths))
idxs = np.random.choice(idxs, size=(25,), replace=False)
images = []

loop over the testing samples

In [38]:
for i in idxs:
	# load the testing image, clone it, and resize it
	image = cv2.imread(testingPaths[i])
	output = image.copy()
	output = cv2.resize(output, (128, 128))
	# pre-process the image in the same manner we did earlier
	image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	image = cv2.resize(image, (200, 200))
	image = cv2.threshold(image, 0, 255,
		cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
	# quantify the image and make predictions based on the extracted features using the last trained Random Forest
	features = quantify_image(image)
	preds = model.predict([features])
	label = le.inverse_transform(preds)[0]
	# draw the colored class label on the output image and add it to the set of output images
	color = (0, 255, 0) if label == "healthy" else (0, 0, 255)
	cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,color, 2)
	images.append(output)

create a montage using 128x128 "tiles" with 5 rows and 5 columns

In [39]:
montage = build_montages(images, (128, 128), (5, 5))[0]

show the output montage

In [57]:
!tar -zcvf parkinson.tgz parkinson.pkl

parkinson.pkl


In [58]:
!ls

parkinson.pkl  parkinsonsmodel.tgz  parkinson.tgz  spiral


In [42]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 14.3 MB/s eta 0:00:01


In [98]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {'url' : 'https://us-south.ml.cloud.ibm.com', 'apikey' : '4pnj8AysNXdvLjGOfoArujv-H1MyQZQDzHStIYxlbD2g'}
client = APIClient(wml_credentials)

In [99]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------------  ------------------------
ID                                    NAME                  CREATED
f448aaf4-a3b2-44e2-b54e-361c44a082b8  parkinsons_detection  2022-11-19T12:54:00.986Z
------------------------------------  --------------------  ------------------------


In [100]:
space_ID = 'f448aaf4-a3b2-44e2-b54e-361c44a082b8'

In [102]:
client.set.default_space(space_ID)

'SUCCESS'

In [103]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [104]:
import sklearn
sklearn.__version__

'1.0.2'

In [105]:
MODEL_NAME = "ParkinsonDiseaseDetection_DeployedModel"
DEPLOYMENT_NAME = "ParkinsonDiseaseDetection"
DEMO_MODEL = model

In [106]:
software_spec_uid = client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [107]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

In [110]:
model_details = client.repository.store_model(
    model = DEMO_MODEL,
    meta_props = model_props,
    training_data = X_train,
    training_target = y_train
)

In [111]:
model_id = client.repository.get_model_id(model_details)
model_id

'becb91f7-0541-4937-ac5e-eea7130a9695'

In [113]:
client.repository.download('becb91f7-0541-4937-ac5e-eea7130a9695','Deployed_Model.tar.gz')

Successfully saved model content to file: 'Deployed_Model.tar.gz'


'/home/wsuser/work/Deployed_Model.tar.gz'

In [117]:
deployment_props = {
    client.deployments.ConfigurationMetaNames.NAME : DEPLOYMENT_NAME,
    client.deployments.ConfigurationMetaNames.ONLINE : {}
}

In [119]:
deployment = client.deployments.create(
    artifact_uid = model_id,
    meta_props = deployment_props
)



#######################################################################################

Synchronous deployment creation for uid: 'becb91f7-0541-4937-ac5e-eea7130a9695' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='64ab6137-5fbe-4afb-b6d3-a4ea9680a971'
------------------------------------------------------------------------------------------------


